In [0]:
import os

account_name = os.getenv('AZURE_STORAGE_ACCOUNT_NAME')
account_key = os.getenv('AZURE_STORAGE_ACCOUNT_KEY')

dbutils.fs.mount(
    source = f"wasbs://bronze@{account_name}.blob.core.windows.net/",
    mount_point = f"/mnt/bronze",
    extra_configs = {f"fs.azure.account.key.{account_name}.blob.core.windows.net": account_key}
)

True

In [0]:
#silver

import os

account_name = os.getenv('AZURE_STORAGE_ACCOUNT_NAME')
account_key = os.getenv('AZURE_STORAGE_ACCOUNT_KEY')

dbutils.fs.mount(
    source = f"wasbs://silver@{account_name}.blob.core.windows.net/",
    mount_point = f"/mnt/silver",
    extra_configs = {f"fs.azure.account.key.{account_name}.blob.core.windows.net": account_key}
)

True

In [0]:
#gold

import os

account_name = os.getenv('AZURE_STORAGE_ACCOUNT_NAME')
account_key = os.getenv('AZURE_STORAGE_ACCOUNT_KEY')

dbutils.fs.mount(
    source = f"wasbs://gold@{account_name}.blob.core.windows.net/",
    mount_point = f"/mnt/gold",
    extra_configs = {f"fs.azure.account.key.{account_name}.blob.core.windows.net": account_key}
)

True

In [0]:
#criar database

spark.sql("CREATE DATABASE IF NOT EXISTS covid")

DataFrame[]

In [0]:
#ler dados da camada bronze

df_covid_br_bronze = spark.read.format('csv').options(header='true', infer_schema='true', delimiter=';').load('dbfs:/mnt/bronze/SRAG_01-06.csv')

In [0]:
#limpeza dados vazios

df_covid_br_silver = df_covid_br_bronze.filter(df_covid_br_bronze.ID_REGIONA.isNotNull())

In [0]:
#transforma arquivos em parquet

df_covid_br_silver.write.format('delta').mode('overwrite').save('/mnt/silver/br_covid')

In [0]:
#camada gold
from pyspark.sql.functions import concat, to_date, year, month

In [0]:
df_covid_br_silver = spark.read.format('delta').load('/mnt/silver/br_covid')

In [0]:
#converte as datas do tipo string para date

df_covid_br_silver = df_covid_br_silver.withColumn('DT_NOTIFIC', to_date(df_covid_br_silver['DT_NOTIFIC']))
df_covid_br_silver = df_covid_br_silver.withColumn('DT_SIN_PRI', to_date(df_covid_br_silver['DT_SIN_PRI']))
df_covid_br_silver = df_covid_br_silver.withColumn('DT_NASC', to_date(df_covid_br_silver['DT_NASC']))

In [0]:
#adicionar coluna data da carga
from pyspark.sql.functions import current_date

df_covid_br_gold = df_covid_br_silver.withColumn('DT_CARGA', current_date())

In [0]:
#renomeando colunas

df_covid_br_gold = df_covid_br_gold.withColumnRenamed("DT_NOTIFIC", "DT_NOTIFICACAO").withColumnRenamed("DT_SIN_PRI", "DT_PRIMEIROS_SINTOMAS").withColumnRenamed("DT_NASC", "DT_NASCIMENTO")

In [0]:
#adicionando colunas YEAR e MONTH

df_covid_br_gold = df_covid_br_gold.withColumn('YEAR', year(df_covid_br_gold['DT_NOTIFICACAO']))
df_covid_br_gold = df_covid_br_gold.withColumn('MONTH', month(df_covid_br_gold['DT_NOTIFICACAO']))

In [0]:
df_covid_br_gold.write.format('delta').mode('overwrite').option("mergeSchema", "true").partitionBy('YEAR', 'MONTH').save('/mnt/gold/br_covid')

In [0]:
#criar tabela agregada

from pyspark.sql.functions import count

selected_columns = ["year", "month", "DT_NOTIFICACAO", "SG_UF_NOT", "ID_MUNICIP", "DT_CARGA"]
df_select_columns_gold = df_covid_br_gold.select(selected_columns)
grouped_df = df_select_columns_gold.groupBy(selected_columns).agg(count("*").alias("count"))

In [0]:
display(grouped_df)

year,month,DT_NOTIFICACAO,SG_UF_NOT,ID_MUNICIP,DT_CARGA,count
2020,1,2020-01-08,AM,MANAUS,2024-08-04,5
2020,1,2020-01-27,SP,MOJI MIRIM,2024-08-04,1
2020,1,2020-01-30,AC,RIO BRANCO,2024-08-04,1
2020,1,2020-01-13,GO,CATALAO,2024-08-04,1
2020,2,2020-02-05,SP,SAO PAULO,2024-08-04,12
2020,1,2020-01-30,MA,SAO LUIS,2024-08-04,2
2020,2,2020-02-17,SP,BOITUVA,2024-08-04,1
2020,2,2020-02-16,PR,LONDRINA,2024-08-04,1
2020,2,2020-02-19,GO,CHAPADAO DO CEU,2024-08-04,1
2020,2,2020-02-20,AM,MANAUS,2024-08-04,2


In [0]:
grouped_df.write.format('delta').mode('overwrite').partitionBy('YEAR', 'MONTH').save('/mnt/gold/br_covid_gold_agg')